In [2]:
import pandas as pd
import cv2
import os

In [6]:
images = []
labels = []
categories = ['sport', 'driving']  # Replace with your actual categories

for category in categories:  # categories you have like 'sports', 'shopping', etc.
    for image in os.listdir(f'{category}'):
        img = cv2.imread(f'{category}/{image}')
        img = cv2.resize(img, (224, 224))  # Resize if necessary
        images.append(img)
        labels.append(category)

In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import numpy as np

labels = np.array(labels)
# y = np.argmax(labels, axis=1)
print(labels)
y= labels

['sport' 'sport' 'sport' 'sport' 'sport' 'sport' 'sport' 'sport' 'sport'
 'sport' 'sport' 'sport' 'sport' 'sport' 'sport' 'sport' 'sport' 'sport'
 'sport' 'sport' 'sport' 'sport' 'sport' 'sport' 'sport' 'sport' 'sport'
 'sport' 'sport' 'sport' 'sport' 'sport' 'sport' 'sport' 'driving'
 'driving' 'driving' 'driving' 'driving' 'driving' 'driving' 'driving'
 'driving' 'driving' 'driving' 'driving' 'driving' 'driving' 'driving'
 'driving' 'driving' 'driving' 'driving' 'driving' 'driving' 'driving'
 'driving' 'driving' 'driving' 'driving' 'driving' 'driving' 'driving'
 'driving']


In [12]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y)

# Normalize image vectors
images = np.array(images) / 255.0

# Split the data
X_train, X_test, y_train, y_test = train_test_split(images, y, test_size=0.2)



In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

In [14]:

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add more layers as necessary

model.add(Flatten())  # Flatten before dense layers
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(len(categories)))  # Output layer: one per category
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [15]:
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.1)


Epoch 1/10
2/2 [==============================] - 5s 2s/step - loss: 6.8541 - accuracy: 0.4889 - val_loss: 15.1060 - val_accuracy: 0.5000
Epoch 2/10
2/2 [==============================] - 2s 895ms/step - loss: 13.3349 - accuracy: 0.6889 - val_loss: 8.6266 - val_accuracy: 0.5000
Epoch 3/10
2/2 [==============================] - 2s 955ms/step - loss: 11.4007 - accuracy: 0.6444 - val_loss: 0.9611 - val_accuracy: 0.8333
Epoch 4/10
2/2 [==============================] - 2s 804ms/step - loss: 3.8814 - accuracy: 0.7778 - val_loss: 0.0626 - val_accuracy: 1.0000
Epoch 5/10
2/2 [==============================] - 2s 938ms/step - loss: 2.7021 - accuracy: 0.8222 - val_loss: 1.4305e-06 - val_accuracy: 1.0000
Epoch 6/10
2/2 [==============================] - 2s 931ms/step - loss: 1.4344 - accuracy: 0.8889 - val_loss: 3.6246 - val_accuracy: 0.6667
Epoch 7/10
2/2 [==============================] - 2s 877ms/step - loss: 3.4989 - accuracy: 0.7556 - val_loss: 0.9088 - val_accuracy: 0.8333
Epoch 8/10
2/2 [

In [16]:
model.evaluate(X_test, y_test)


1/1 [==============================] - 0s 134ms/step - loss: 2.0154 - accuracy: 0.5385


[2.0153746604919434, 0.5384615659713745]

In [28]:
import requests

def predict_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224)) / 255.0
    prediction = model.predict(np.array([img]))
    return label_encoder.inverse_transform([np.argmax(prediction)])[0]

def predict_image_from_url(image_url):
    response = requests.get(image_url)
    if response.status_code == 200:
        image_data = np.frombuffer(response.content, dtype=np.uint8)
        img = cv2.imdecode(image_data, cv2.IMREAD_COLOR)
        if img is not None:
            img = cv2.resize(img, (224, 224)) / 255.0
            prediction = model.predict(np.array([img]))
            return label_encoder.inverse_transform([np.argmax(prediction)])[0]
        else:
            return "Failed to decode image"
    else:
        return "Failed to download image"

predict_image_from_url('https://location-voitures-essaouira.com/wp-content/uploads/2015/05/location-voitures-essaouira-rent-car-essaouira%20(2).png')

1/1 [==============================] - 0s 52ms/step


'sport'

In [ ]:
plt.scatter(y_test,y_pred)
plt.grid